In [20]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# from HelperFunctions import minibatch 
%reload_ext autoreload
%autoreload 
from HelperFunctions import minibatch 

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
ols = LinearRegression()
ridge = Ridge()
lasso  = Lasso()
elasticnet = ElasticNet()

from sklearn import tree
regressor = tree.DecisionTreeRegressor()

from sklearn import ensemble
randomForest = ensemble.RandomForestRegressor()
bagging = ensemble.BaggingRegressor()
gbm = ensemble.GradientBoostingRegressor()
abr = ensemble.AdaBoostRegressor()

import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [14]:
df=pd.read_csv('lean_df_4.csv',index_col='id')

/home/auscheng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (34,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/auscheng/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [31]:
mybatch=minibatch(df)

In [32]:
testdf=mybatch.takeout('2007-06','2007-07')
testdf

,loan_amnt,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,pub_rec,revol_util,total_acc,initial_list_status,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,total_rec_late_fee,collection_recoveries,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,open_act_il,open_il_12m,mths_since_rcnt_il,il_util,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_bc_sats,num_tl_120dpd_2m,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_il_high_credit_limit,fico,revol_frac,install_frac,mort_frac,card_frac,active_card_frac,active_revol_frac,open_revol_frac,good_acc_frac,fully_invested
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
87023,7500.0,36 months,13.75,E,E2,Evergreen Center,< 1 year,OWN,22000.0,Not Verified,2007-06-01,Fully Paid,debt_consolidation,027xx,MA,14.29,1.0,Oct-2003,0.0,11.0,0.0,0.0,51.5,8.0,f,0.0,9195.263334,7500.00,1695.26,0.00,0.00,Jun-2010,256.59,NaN,Mar-2019,NaN,NaN,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,662.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
88637,6000.0,36 months,10.59,C,C2,Yale University,< 1 year,RENT,20000.0,Not Verified,2007-06-01,Fully Paid,debt_consolidation,065xx,CT,12.90,0.0,Jan-1996,4.0,45.0,0.0,0.0,66.0,6.0,f,0.0,7029.871272,6000.00,1029.87,0.00,0.00,Jun-2010,197.36,Jul-2010,Oct-2014,NaN,NaN,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,697.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
88046,4400.0,36 months,9.64,B,B4,Brick Township board of education,2 years,MORTGAGE,30000.0,Not Verified,2007-06-01,Fully Paid,debt_consolidation,087xx,NJ,3.72,0.0,Jul-2004,0.0,0.0,0.0,0.0,63.5,5.0,f,0.0,5084.724868,4400.00,684.72,0.00,0.00,Jun-2010,143.28,Jul-2010,May-2018,NaN,NaN,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,697.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
85961,1200.0,36 months,9.01,B,B2,Classic Components,< 1 year,RENT,36000.0,Not Verified,2007-06-01,Fully Paid,other,787xx,TX,3.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,0.0,1373.990771,1200.00,173.99,0.00,0.00,Jul-2010,39.05,Jul-2010,Mar-2019,NaN,NaN,Individual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,707.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
85818,5000.0,36 months,11.22,C,C4,Compensation Solutions,< 1 year,RENT,28000.0,Not Verified,2007-06-01,Fully Paid,other,078xx,NJ,3.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,0.0,5858.468448,5000.00,828.47,30.00,0.00,Apr-2010,149.43,Apr-2010,Jan-2017,NaN,NaN,Individual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,672.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
85781,1400.0,36 months,10.91,C,C3,"Stanford University Libraries, LOCKSS Project",< 1 year,RENT,40000.0,Not Verified,2007-06-01,Fully Paid,other,652xx,MO,8.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,0.0,1663.240931,1400.00,248.24,15.00,0.00,Jul-2010,1.78,Aug-2010,Sep-2014,NaN,NaN,Individual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,672.0,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
testdf

In [ ]:
def FUNC1(df1,df2,df3,df4):
    

In [ ]:
param_grid = {
    'max_depth' : range(5,20),\
    'max_features': range(5,20)
}
randomForest = ensemble.RandomForestRegressor()
grid_search_forest = GridSearchCV(ensemble.RandomForestRegressor(n_estimators=100,criterion="mse",\
                                                                 min_samples_leaf=2,min_samples_split=2,\
                                                                 bootstrap=True,oob_score = True,n_jobs=-1),\
                                  param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1,refit=True)
%time grid_search_forest.fit(trainX, trainY)

# Print results    
print('Best parameters: {}'.format(grid_search_forest.best_params_))
print('Best MSE Score: {0:.3f}'.format(-grid_search_forest.best_score_)) # NOTE: it is negated! (Sklearn convention)
print('MSE Score on training data set (using best parameters): {0:.3f}'.format(-grid_search_forest.score(trainX, trainY)))
print('MSE Score on test data set (using best parameters): {0:.3f}'.format(-grid_search_forest.score(testX, testY)))
print('RMSE in $$ on test data set (using best parameters): {0:.1f}'.format(\
      np.mean((np.exp(grid_search_forest.predict(testX))-np.exp(testY))**2)**0.5))

In [ ]:
xplot=list(param_grid['max_depth'])
yplot=list(param_grid['max_features'])
zplot=-1*grid_search_forest.cv_results_['mean_test_score'].reshape(len(yplot),-1) # NOTE: it is negated! (Sklearn convention)
plt.pcolor(xplot,yplot,zplot,cmap='RdBu')
plt.colorbar();
plt.xlabel('Maximum Tree Depth');
plt.ylabel('Max Features to Consider for Each Split');

In [ ]:
number_estimators = range(10,110,10)
train_error = []
test_error = []
for i in number_estimators:
    randomForest.set_params(n_estimators=i,criterion="mse",bootstrap=True,oob_score = False,n_jobs=-1,random_state=0,\
                       max_depth=grid_search_forest.best_params_['max_depth'],max_features=grid_search_forest.best_params_['max_features'])
    randomForest.fit(trainX, trainY)
    train_error.append(metrics.mean_squared_error(trainY, randomForest.predict(trainX))) 
    test_error.append(metrics.mean_squared_error(testY, randomForest.predict(testX))) 
import matplotlib.pyplot as plt
plt.plot(number_estimators, train_error, c='red', label='training error')
plt.plot(number_estimators, test_error, c='blue', label='test error')
plt.ylabel('MSE')
plt.xlabel('Number of Trees')
plt.legend()
plt.show()

In [ ]:
#visualize best predictors
randomForest.set_params(n_estimators=100,criterion="mse",bootstrap=True,oob_score = False,n_jobs=-1,random_state=0,\
                       max_depth=grid_search_forest.best_params_['max_depth'],max_features=grid_search_forest.best_params_['max_features'])
randomForest.fit(trainX,trainY)
feature_importance=pd.Series(randomForest.feature_importances_,index=trainX.columns)
feature_importance.sort_values(ascending=False,inplace=True)
plt.figure(figsize=(10,10))
sns.barplot(x=feature_importance.index[0:20],y=feature_importance.values[0:20])
plt.xticks(rotation=90)